In [83]:
import surprise
import numpy as np
import pandas as pd

from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import KNNWithMeans
from surprise import SlopeOne
from surprise.model_selection import cross_validate
from tqdm import tqdm_notebook, tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from surprise import accuracy
from surprise.model_selection import KFold
from multiprocessing import Pool
tqdm = tqdm_notebook

In [90]:
# - try implicit library

In [2]:
file_path = 'data/data_train.csv'


# I have to convert the 
def deal_line(line):
    pos, rating = line.split(',')
    row, col = pos.split("_")
    row = row.replace("r", "")
    col = col.replace("c", "")
    return int(row), int(col), float(rating)

def read_txt(path):
    """read text file from path."""
    with open(path, "r") as f:
        return f.read().splitlines()
    
lines = read_txt(file_path)[1:]
data = [deal_line(line) for line in lines]

with open('data/kiru.csv', 'w') as f:
    for item in data:
        f.write("{},{},{}\n".format(item[0], item[1], item[2]))

# path to dataset file
file_path = 'data/kiru.csv'
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_file(file_path, reader=reader)

In [9]:
def call_algo(i):
    trainset, testset = i
    # train and test algorithm.
    algo = SlopeOne()
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)
    return (algo, rmse)
        
def predict():
    with Pool(10) as p:
        # define a cross-validation iterator
        kf = KFold(n_splits=10)
        return p.map(call_algo, kf.split(data))

In [11]:
predictions = predict()
best_model_slope_one = min(predictions, key=lambda b: b[1])

RMSE: 1.0050
RMSE: 0.9966
RMSE: 0.9987
RMSE: 0.9988
RMSE: 0.9977
RMSE: 1.0016
RMSE: 0.9966
RMSE: 0.9987
RMSE: 1.0019
RMSE: 0.9996


In [13]:
def call_algo_svd(i):
    trainset, testset = i
    # train and test algorithm.
    algo = SVD()
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)
    return (algo, rmse)
        
def predict_svd():
    with Pool(10) as p:
        # define a cross-validation iterator
        kf = KFold(n_splits=10)
        return p.map(call_algo_svd, kf.split(data))

In [14]:
predictions_svd = predict_svd()
best_model_svd = min(predictions_svd, key=lambda b: b[1])

RMSE: 1.0244
RMSE: 1.0277
RMSE: 1.0197
RMSE: 1.0239
RMSE: 1.0226
RMSE: 1.0215
RMSE: 1.0263
RMSE: 1.0230
RMSE: 1.0204
RMSE: 1.0209


In [68]:
predictions = avd.test(testset)
# Compute and print Root Mean Squared Error
accuracy.rmse(predictions, verbose=True)

RMSE: 1.0257


1.0257002670379434

Lets train a linear regression to find the best coefficient

In [88]:
def combine(first, second):
    kf = KFold(n_splits=10)
    weights = []
    for trainset, testset in tqdm(kf.split(data), desc="split"):
        print("Do one split")
        input_for_linear = []
        for each in tqdm(trainset.all_ratings(), desc="Process rating of split"):
            pred1 = first.predict(str(each[0]), str(each[1])).est
            pred2 = second.predict(str(each[0]), str(each[1])).est
            output = [pred1, pred2]
            input_for_linear.append(output)
        
        y = list(map(lambda x: x[2], trainset.all_ratings()))
        eg = LinearRegression().fit(input_for_linear, y)
        
        predictions = []
        c = eg.coef_
        for i, b in enumerate(input_for_linear):
           # p = b[0] * c[0] + b[1] + c[1]
            p = b[0] * 0.5 + b[1] + 0.5
            predictions.append(p)
            
        error = np.sqrt(mean_squared_error(y, predictions))
        weights.append((error, c))
        
    return min(weights, key=lambda b: b[0])
        
best_weights = combine(best_model[0], best_model_svd[0])

Do one split


Do one split


Do one split


Do one split


Do one split


Do one split


Do one split


Do one split


Do one split


Do one split


In [87]:
print(best_weights)

(1.3056150432848417, array([ 0.00796198, -0.02192079]))


In [89]:
print(best_weights)

(2.391156809375227, array([ 0.02076612, -0.02743119]))


In [77]:
def create_submission_file(first, second, w1, w2):
    lines = read_txt('data/sample_submission.csv')[1:]
    data = [deal_line(line) for line in lines]

    predictions = []
    for each in tqdm(data):
        pred1 = first.predict(str(each[0]), str(each[1])).est
        pred2 = second.predict(str(each[0]), str(each[1])).est
        predictions.append((each[0], each[1], pred1 * w1 + pred2 * w2 ))

    with open('data/our_pred.csv', 'w') as f:
        f.write("Id,Prediction\n")
        for item in tqdm(predictions):
            f.write("r{}_c{},{}\n".format(item[0], item[1], int(round(item[2]))))

In [66]:
create_submission_file(avd, algo)

100%|██████████| 1176952/1176952 [00:02<00:00, 424605.17it/s]


In [46]:
accuracy.rmse(algo, verbose=True)

TypeError: 'SlopeOne' object is not iterable